In [35]:
import numpy as np
import cv2
try:
    from PIL import Image
except ImportError:
    import Image
import pytesseract
 
def hello_whale():
    img_rgb = cv2.imread('test1.jpg', cv2.IMREAD_UNCHANGED)
    match_msq_tracker(img_rgb)
    match_quest_tracker(img_rgb)
    
    
def match_msq_tracker(img_rgb):
    print("Matching msq tracker")
    img = cv2.cvtColor(np.asarray(img_rgb), cv2.COLOR_BGR2GRAY)
    template = cv2.imread('msq_box.jpg', 0)
    w, h = template.shape[::-1]
    
    res = cv2.matchTemplate(img, template, cv2.TM_CCOEFF_NORMED)
    threshold = 0.8
    
    loc = np.where( res >= threshold)
    res = list(zip(*loc[::-1]))
    
    print(len(res), "results")
    for pt in zip(*loc[::-1]):
        y1 = max(pt[1], 0)
        y2 = min(pt[1] + h, img.shape[::-1][1])
        x1 = max(pt[0], 0)
        x2 = min(pt[0] + w, img.shape[::-1][0])
        
        cv2.rectangle(img_rgb, (x1, y1), (x2, y2), (0,0,255), 2)
        cropped = img[y1:y2, x1:x2]
        quest_text = pytesseract.image_to_string(cropped).strip().lstrip().rstrip()
        print(quest_text)
        
    cv2.imshow("quest_tracker", img_rgb)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    
    
def match_quest_tracker(img_rgb):
    print("Matching quest tracker")
    img = cv2.cvtColor(np.asarray(img_rgb), cv2.COLOR_BGR2GRAY)
    template = cv2.imread('msq_icon.png', 0)
    w, h = template.shape[::-1]
    
    res = cv2.matchTemplate(img, template, cv2.TM_CCOEFF_NORMED)
    threshold = 0.8
    
    loc = np.where( res >= threshold)
    res = list(zip(*loc[::-1]))
    
    print(len(res), "results")
    for pt in zip(*loc[::-1]):
        y1 = max(pt[1], 0)
        y2 = min(pt[1] + h, img.shape[::-1][1])
        x1 = max(pt[0] - 350, 0)
        x2 = min(pt[0] + int(w/8), img.shape[::-1][0])
        
        cv2.rectangle(img_rgb, (x1, y1), (x2, y2), (0,0,255), 2)
        cropped = img[y1:y2, x1:x2]
        quest_text = pytesseract.image_to_string(cropped).strip().lstrip().rstrip()
        print(quest_text)
        
    cv2.imshow("quest_tracker", img_rgb)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

    
 

hello_whale()

Matching msq tracker
44 results
Ce
Operation Archon
a
Operation Archon
a
Operation Archon
a
yy Operation Archon
a
y Operation Archon
a
y Operation Archon
a
yi Operation Archon
a
y Operation Archon
a
y Operation Archon
ree era pers
y Operation Archon
em
Operation Archon
a
Operation Archon
EST STR RR
Operation Archon
9
Operation Archon
a
j Operation Archon
SE STRESS a
Operation Archon
a
j, Operation Archon
— SS

» Operation Archon
» Operation Archon
) Operation Archon
Operation Archon !
) Operation Archon '
) Operation Archon '
| Operation Archon ’
Operation Archon ’
Operation Archon ’
Ol hE
Operation Archon
Operation Archon
Operation Archon
Operation Archon
Operation Archon
Operation Archon


)peration Archon j
)peration Archon ]
)peration Archon ]
)peration Archon ]


ion Archon
peration Archon I
peration Archon I
,eration Archon j
Matching quest tracker
5 results
Operation Archon
Operation Archon
Operation Archon
Operation Archon
Operation Archon
